In [1]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.4     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



[1] "hnsc-data"          "notebook9d39ba3a4b"

In [2]:
lihc_deg_3000 = read.table(file = "../input/notebook9d39ba3a4b/nhsc_degnames_3000.txt", sep = "\t", header = TRUE, row.names = 1)$x
write.table(lihc_deg_3000, file = "./nhsc_degnames_3000_final.txt", sep = "\t")

In [3]:
if (!requireNamespace("BiocManager", quietly = TRUE))
  install.packages("BiocManager")

if (!require("SummarizedExperiment", character.only=T, quietly=F)) 
{
  BiocManager::install("SummarizedExperiment")
  library("SummarizedExperiment")
}

Loading required package: SummarizedExperiment

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘SummarizedExperiment’”
Bioconductor version 3.10 (BiocManager 1.30.10), R 3.6.3 (2020-02-29)

Installing package(s) 'SummarizedExperiment'

also installing the dependencies ‘zlibbioc’, ‘XVector’, ‘BiocParallel’, ‘GenomeInfoDbData’, ‘GenomicRanges’, ‘Biobase’, ‘DelayedArray’, ‘S4Vectors’, ‘IRanges’, ‘GenomeInfoDb’


Old packages: 'astsa', 'backports', 'BART', 'bayestestR', 'BMA', 'broom',
  'cld2', 'clusterGeneration', 'colourvalues', 'compareGroups', 'config',
  'd3Tree', 'data.table', 'Deriv', 'DescTools', 'doFuture', 'effectsize',
  'emmeans', 'epiR', 'ExPanDaR', 'FactoInvestigate', 'FactoMineR', 'fda',
  'feasts', 'fitdistrplus', 'flextable', 'foghorn', 'forestplot', 'fpc',
  'future', 'geepack', 'ggalluvial', 'ggeffects', 'gghighlight', 'ggrepel',
  'ggstance', 'ggtext', 'ggvis', 'gitcreds', 'googleAuthR'

In [4]:
#library(TCGAbiolinks)
library(SummarizedExperiment)

load("../input/hnsc-data/hnsc_rnaseq_full_counts.RData")

In [5]:
ens_gnames_tcga = toupper(rownames(assay(hnsc.exp)))
length(unique(ens_gnames_tcga))
# 56493
length(unique(toupper(ens_gnames_tcga)))
# 56493
length(ens_gnames_tcga)
# 56493

[1] 56493

[1] 56493

[1] 56493

In [6]:
if (!require("biomaRt", character.only=T, quietly=F)) {
  BiocManager::install("biomaRt")
  library("biomaRt")
}

Loading required package: biomaRt

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘biomaRt’”
Bioconductor version 3.10 (BiocManager 1.30.10), R 3.6.3 (2020-02-29)

Installing package(s) 'biomaRt'

also installing the dependencies ‘AnnotationDbi’, ‘BiocFileCache’


Old packages: 'astsa', 'backports', 'BART', 'bayestestR', 'BMA', 'broom',
  'cld2', 'clusterGeneration', 'colourvalues', 'compareGroups', 'config',
  'd3Tree', 'data.table', 'Deriv', 'DescTools', 'doFuture', 'effectsize',
  'emmeans', 'epiR', 'ExPanDaR', 'FactoInvestigate', 'FactoMineR', 'fda',
  'feasts', 'fitdistrplus', 'flextable', 'foghorn', 'forestplot', 'fpc',
  'future', 'geepack', 'ggalluvial', 'ggeffects', 'gghighlight', 'ggrepel',
  'ggstance', 'ggtext', 'ggvis', 'gitcreds', 'googleAuthR', 'googlePolylines',
  'gridGraphics', 'gridtext', 'HardyWeinberg', 'here', 'hildareadR',
  'htmlwidgets', 'hunspell', 'ifultools', 'insight', 'ipmi

In [7]:
require(biomaRt)
mart <- useMart(
  biomart = 'ENSEMBL_MART_ENSEMBL',
  dataset = 'hsapiens_gene_ensembl')

annotLookup <- getBM(mart = mart,
                     attributes = c('ensembl_gene_id', 'entrezgene_id', 'gene_biotype', 'hgnc_symbol'),
                     uniqueRows = TRUE,useCache = FALSE)

In [8]:
length(which(ens_gnames_tcga %in% toupper(annotLookup$ensembl_gene_id)))
#[1] 56457
length(which(toupper(annotLookup$ensembl_gene_id) %in% ens_gnames_tcga))
#[1] 56659
annotLookup_tcga = annotLookup[which(toupper(annotLookup$ensembl_gene_id) %in% ens_gnames_tcga),]
length(annotLookup_tcga$ensembl_gene_id)
# [1] 56659
length(unique(annotLookup_tcga$ensembl_gene_id))
# [1] 56457
length(which(annotLookup_tcga$hgnc_symbol == ""))
# [1] 18694
# View(annotLookup_tcga)
length(which(is.na(annotLookup_tcga$hgnc_symbol)))
# 0
dim(annotLookup_tcga)
# [1] 56659     4
annotLookup_tcga_ogs = annotLookup_tcga[-which(annotLookup_tcga$hgnc_symbol == ""),]
dim(annotLookup_tcga_ogs)
# [1] 37965     4
length(toupper(annotLookup_tcga_ogs$hgnc_symbol))
# [1] 37965
length(unique(toupper(annotLookup_tcga_ogs$hgnc_symbol)))
# [1] 37755

[1] 56448

[1] 56595

[1] 56595

[1] 56448

[1] 18291

[1] 0

[1] 56595     4

[1] 38304     4

[1] 38304

[1] 38149

In [9]:
df_gene_counts_ogs = as.data.frame(table(annotLookup_tcga_ogs$hgnc_symbol))
df1_gene_counts_ogs = df_gene_counts_ogs[which(df_gene_counts_ogs$Freq > 1),]
# View(df1_gene_counts_ogs)
ens_gnames_tcga_ogs = unique(toupper(annotLookup_tcga_ogs$ensembl_gene_id))
length(ens_gnames_tcga_ogs)
# [1] 37765
df2_gene_counts_ogs = as.data.frame(table(annotLookup_tcga_ogs$ensembl_gene_id))
dim(df2_gene_counts_ogs)
# [1] 37765     2
df3_gene_counts_ogs = df2_gene_counts_ogs[which(df2_gene_counts_ogs$Freq > 1),]
dim(df3_gene_counts_ogs)
# [1] 163   2
# View(df3_gene_counts_ogs)
lihc_count_ogs = assay(hnsc.exp)[which(ens_gnames_tcga_ogs %in% toupper(rownames(assay(hnsc.exp)))),]
dim(lihc_count_ogs)
# [1] [1] 37765   424

[1] 38158

[1] 38158     2

[1] 120   2

[1] 38158   546

In [10]:
lihc_deg_3000 = read.table("../input/notebook9d39ba3a4b/nhsc_degnames_3000.txt", sep = "\t", header = TRUE)
lihc_deg_3000 = as.character(lihc_deg_3000$x)

lihc_deg_3000 = toupper(lihc_deg_3000)

length(lihc_deg_3000)

# [1] 3000


length(which(lihc_deg_3000 %in% toupper(rownames(lihc_count_ogs))))
# [1] 2192

length(which(toupper(rownames(lihc_count_ogs)) %in% lihc_deg_3000))
# [1] 2192

length(which(rownames(lihc_count_ogs) %in% lihc_deg_3000))
# [1] 2192

length(which(lihc_deg_3000 %in% rownames(lihc_count_ogs)))
# [1] 2192


length(which(lihc_deg_3000 %in% toupper(rownames(lihc_count_ogs))))
# [1] 2192


[1] 1393

[1] 1096

[1] 1096

[1] 1096

[1] 1096

[1] 1096

In [11]:
which(lihc_deg_3000 %in% toupper(rownames(lihc_count_ogs)))

lihc_deg_3000[which(lihc_deg_3000 %in% toupper(rownames(lihc_count_ogs)))]

lihc_deg_3000_keep = lihc_deg_3000[which(lihc_deg_3000 %in% toupper(rownames(lihc_count_ogs)))]

annotLookup_tcga_deg_3000 = annotLookup[which(toupper(annotLookup$ensembl_gene_id) %in% lihc_deg_3000_keep),]
# View(annotLookup_tcga_deg_3000)

length(which(annotLookup_tcga_deg_3000$hgnc_symbol == ""))
# [1] 258


[1]    4    5    6    7    8    9   10   12   15   16   18   22   23   25
  [15]   26   28   30   31   32   33   34   36   37   38   40   43   44   46
  [29]   47   48   49   50   51   52   53   54   56   57   58   59   60   61
  [43]   63   64   65   67   68   69   70   71   72   76   78   79   80   82
  [57]   85   87   88   89   90   91   93   96   97   98   99  100  101  102
  [71]  103  104  105  106  107  109  110  111  112  113  114  115  118  119
  [85]  120  121  122  123  124  125  126  128  131  132  133  134  135  137
  [99]  138  139  140  142  144  145  146  147  150  151  152  153  154  155
 [113]  156  158  159  160  161  162  163  164  167  168  169  170  171  173
 [127]  174  175  176  178  180  181  182  183  184  185  187  188  189  190
 [141]  192  193  194  195  196  197  198  199  202  203  207  208  209  211
 [155]  212  213  214  215  217  219  220  222  224  226  228  229  232  234
 [169]  235  237  239  240  241  242  244  245  246  247  248  249  250  251
 [183]  253  255  256  257  260  261  262  263  264  265  266  267  270  274
 [197]  275  276  277  279  280  282  285  286  287  289  290  291  292  293
 [211]  295  297  298  299  302  304  306  307  308  309  311  312  314  315
 [225]  316  319  320  321  322  323  325  327  331  335  336  337  338  340
 [239]  341  342  344  345  346  347  349  350  351  352  353  354  360  363
 [253]  364  366  370  372  373  376  377  379  380  381  382  383  384  385
 [267]  386  389  390  391  392  393  394  395  396  397  399  400  401  402
 [281]  404  405  406  408  409  410  411  413  414  416  417  418  419  420
 [295]  421  422  425  426  427  428  429  430  431  433  434  435  436  439
 [309]  440  441  442  443  444  446  447  448  449  450  451  452  453  454
 [323]  455  456  457  459  462  463  464  465  469  471  472  473  474  476
 [337]  479  480  481  482  483  484  487  488  492  493  494  496  497  498
 [351]  499  501  502  504  506  507  509  511  512  513  514  516  518  522
 [365]  525  526  527  528  529  530  531  532  533  535  536  539  540  544
 [379]  545  546  547  550  551  552  553  555  557  558  561  562  563  565
 [393]  566  567  570  571  573  574  575  578  579  580  581  582  584  585
 [407]  586  587  590  591  592  593  594  595  597  599  602  603  604  605
 [421]  607  608  610  612  613  614  615  616  617  619  624  625  626  627
 [435]  628  629  631  632  634  635  636  639  640  641  643  644  645  646
 [449]  647  648  649  653  654  656  657  658  659  660  661  662  663  664
 [463]  665  666  669  674  675  677  678  680  682  683  685  687  688  690
 [477]  691  692  693  694  695  697  698  699  700  701  702  703  704  705
 [491]  707  708  709  710  711  712  713  715  716  718  719  721  722  723
 [505]  724  725  726  730  731  732  733  734  735  736  737  739  740  741
 [519]  743  744  747  749  751  752  754  756  757  758  759  760  761  764
 [533]  765  767  768  769  770  771  772  773  775  776  777  778  779  781
 [547]  782  783  784  785  786  787  790  791  792  793  794  795  796  797
 [561]  799  800  801  804  806  808  811  812  814  815  816  817  818  820
 [575]  822  824  825  826  827  828  829  830  831  832  833  835  836  840
 [589]  841  842  843  844  845  846  848  849  850  851  852  853  854  856
 [603]  858  859  860  861  862  863  864  866  867  868  869  870  871  872
 [617]  873  874  875  877  878  879  880  881  882  883  885  886  887  888
 [631]  890  891  892  893  894  895  896  897  898  899  900  901  902  905
 [645]  906  907  908  909  910  911  912  913  914  915  916  919  920  921
 [659]  922  924  925  926  927  928  929  930  931  932  933  934  935  936
 [673]  937  938  940  941  942  943  944  945  946  947  948  949  950  952
 [687]  953  954  955  957  958  959  960  962  963  964  965  966  967  968
 [701]  969  970  971  972  973  974  975  976  977  978  979  980  981  982
 [715]  983  984  985  987  988  989  990  991  992  993  996  997  998  999

[1] "ENSG00000248131" "ENSG00000244040" "ENSG00000164438" "ENSG00000123584"
   [5] "ENSG00000231431" "ENSG00000185247" "ENSG00000198681" "ENSG00000233455"
   [9] "ENSG00000229967" "ENSG00000181433" "ENSG00000147381" "ENSG00000180178"
  [13] "ENSG00000105278" "ENSG00000198883" "ENSG00000157060" "ENSG00000249069"
  [17] "ENSG00000228636" "ENSG00000185686" "ENSG00000145721" "ENSG00000236516"
  [21] "ENSG00000240280" "ENSG00000239542" "ENSG00000142182" "ENSG00000102195"
  [25] "ENSG00000245870" "ENSG00000089116" "ENSG00000131650" "ENSG00000159184"
  [29] "ENSG00000187714" "ENSG00000077935" "ENSG00000176919" "ENSG00000153303"
  [33] "ENSG00000221867" "ENSG00000238040" "ENSG00000187806" "ENSG00000237118"
  [37] "ENSG00000138356" "ENSG00000159217" "ENSG00000231453" "ENSG00000174948"
  [41] "ENSG00000241084" "ENSG00000205634" "ENSG00000187855" "ENSG00000139515"
  [45] "ENSG00000039987" "ENSG00000197172" "ENSG00000177359" "ENSG00000170689"
  [49] "ENSG00000238098" "ENSG00000184502" "ENSG00000118596" "ENSG00000163825"
  [53] "ENSG00000165556" "ENSG00000232508" "ENSG00000147889" "ENSG00000171658"
  [57] "ENSG00000231505" "ENSG00000037965" "ENSG00000170178" "ENSG00000143382"
  [61] "ENSG00000188269" "ENSG00000159261" "ENSG00000234041" "ENSG00000229400"
  [65] "ENSG00000221887" "ENSG00000183166" "ENSG00000125337" "ENSG00000229522"
  [69] "ENSG00000236961" "ENSG00000196395" "ENSG00000248498" "ENSG00000169548"
  [73] "ENSG00000151834" "ENSG00000205436" "ENSG00000231240" "ENSG00000130822"
  [77] "ENSG00000224854" "ENSG00000244280" "ENSG00000188816" "ENSG00000230750"
  [81] "ENSG00000213468" "ENSG00000197757" "ENSG00000217236" "ENSG00000197993"
  [85] "ENSG00000147378" "ENSG00000236449" "ENSG00000226856" "ENSG00000173805"
  [89] "ENSG00000176049" "ENSG00000222533" "ENSG00000248103" "ENSG00000224194"
  [93] "ENSG00000168671" "ENSG00000225146" "ENSG00000243008" "ENSG00000232197"
  [97] "ENSG00000235034" "ENSG00000124678" "ENSG00000232383" "ENSG00000241418"
 [101] "ENSG00000142619" "ENSG00000246876" "ENSG00000132357" "ENSG00000230316"
 [105] "ENSG00000175894" "ENSG00000164850" "ENSG00000248807" "ENSG00000180806"
 [109] "ENSG00000185904" "ENSG00000238840" "ENSG00000215612" "ENSG00000235621"
 [113] "ENSG00000148408" "ENSG00000230061" "ENSG00000196421" "ENSG00000205704"
 [117] "ENSG00000118156" "ENSG00000171532" "ENSG00000242522" "ENSG00000161860"
 [121] "ENSG00000149243" "ENSG00000107159" "ENSG00000244757" "ENSG00000135248"
 [125] "ENSG00000202379" "ENSG00000164651" "ENSG00000101883" "ENSG00000232188"
 [129] "ENSG00000124657" "ENSG00000174469" "ENSG00000130487" "ENSG00000120068"
 [133] "ENSG00000150276" "ENSG00000099953" "ENSG00000182795" "ENSG00000237402"
 [137] "ENSG00000231568" "ENSG00000200879" "ENSG00000249001" "ENSG00000233384"
 [141] "ENSG00000066923" "ENSG00000189366" "ENSG00000227094" "ENSG00000234828"
 [145] "ENSG00000106031" "ENSG00000230838" "ENSG00000240990" "ENSG00000236045"
 [149] "ENSG00000181355" "ENSG00000182912" "ENSG00000130294" "ENSG00000111961"
 [153] "ENSG00000146197" "ENSG00000130540" "ENSG00000232455" "ENSG00000213590"
 [157] "ENSG00000240498" "ENSG00000179399" "ENSG00000204528" "ENSG00000224184"
 [161] "ENSG00000006606" "ENSG00000199426" "ENSG00000236670" "ENSG00000176273"
 [165] "ENSG00000223631" "ENSG00000128713" "ENSG00000168306" "ENSG00000224271"
 [169] "ENSG00000214429" "ENSG00000167414" "ENSG00000043355" "ENSG00000234838"
 [173] "ENSG00000070404" "ENSG00000226816" "ENSG00000168065" "ENSG00000235047"
 [177] "ENSG00000167281" "ENSG00000232775" "ENSG00000232233" "ENSG00000154277"
 [181] "ENSG00000181803" "ENSG00000214946" "ENSG00000184368" "ENSG00000205307"
 [185] "ENSG00000240457" "ENSG00000228573" "ENSG00000226472" "ENSG00000161031"
 [189] "ENSG00000246985" "ENSG00000127129" "ENSG00000224034" "ENSG00000228510"
 [193] "ENSG00000226468" "ENSG00000006377" "ENSG00000163239" "ENSG00000233064"
 [197] "ENSG00000115297" "ENSG00000196188" "ENSG00000235890" "ENSG00000100253"
 [201] "ENSG00000230798" "ENSG00000050767" "ENSG00000

[1] 94

In [12]:
which(table(annotLookup_tcga_deg_3000$hgnc_symbol) > 1)

which(table(annotLookup_tcga_deg_3000$hgnc_symbol) > 1)

#              CRHR1     CRNDE   GAGE12J    GAGE2A LINC00511 LINC00595 LINC00856 LINC02802 
#        1       362       366       640       641       940       942       952      1063 
#  LNCAROD     RFLNA   STEAP1B 
#     1072      1488      1707          
          

DDT    MCL1   SCNM1   SPSB2 STAG3L4  TBC1D7 TRABD2A 
      1     203     480     767     836     842     861     907

DDT    MCL1   SCNM1   SPSB2 STAG3L4  TBC1D7 TRABD2A 
      1     203     480     767     836     842     861     907

In [13]:
df_gene_counts_deg_ogs = as.data.frame(table(annotLookup_tcga_deg_3000$hgnc_symbol))
# View(df_gene_counts_deg_ogs)
df1_gene_counts_deg_ogs = df_gene_counts_deg_ogs[which(df_gene_counts_deg_ogs$Freq > 1),]
# View(df1_gene_counts_deg_ogs)



# View(annotLookup_tcga_deg_3000)


In [14]:
annotLookup_tcga_deg_3000_keep = annotLookup_tcga_deg_3000[-which(annotLookup_tcga_deg_3000$hgnc_symbol == ""),]
lihc_deg_3000_keep_final = annotLookup_tcga_deg_3000_keep$ensembl_gene_id

In [15]:
write.table(annotLookup_tcga_deg_3000, file = "./annotLookup_tcga_deg_3000_2192g.txt", sep = "\t", row.names = TRUE, col.names = NA)
write.table(annotLookup_tcga_deg_3000_keep, file = "./annotLookup_tcga_deg_3000_1948g_keep.txt", sep = "\t", row.names = TRUE, col.names = NA)


In [16]:
write.table(lihc_deg_3000_keep, file = "./hnsc_deg_3000_2192g_keep.txt", sep = "\t", row.names = TRUE, col.names = NA)
write.table(lihc_deg_3000_keep_final, file = "./hnsc_deg_3000_keep_final_1948g.txt", sep = "\t", row.names = TRUE, col.names = NA)


In [17]:
rm(hnsc.exp, lihc_count_ogs)
save.image("./ogs_hnsc_deg_3000_1948g.RData")